### User OGS game data dump

**This notebook should be run first before any other utilities, as other utilities depend on the game data dump generated by this tool.**

This notebook generates a raw .json file of game information for games played with your OGS account.
It only tracks information *about* each game, and does not dump the games themselves.

After running this notebook, there are the following extensions.
* To clean up the resulting game data dump for statistics/data science applications, run `stats_clean.ipynb`.
* To obtain a dump of the game records, run `games_dump.ipynb`.

In [ ]:
import requests
import json
import time
import random

Before proceeding, register an OAuth 2.0 application at https://online-go.com/oauth2/applications/.
Copy the client id and client secret strings before saving.
For the other fields I used client type 'public' and authorization type 'password'.

In [ ]:
# keys
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

# used to authorize access to account-specific API calls
username = "YOUR_OGS_USERNAME"
password = "YOUR_OGS_PASSWORD"

In [ ]:
# Obtaining authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={"grant_type":"password","username":username,"password":password,"client_id":client_id,"client_secret":client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

# Set endpoint, request first page of results
endpoint = "https://online-go.com/api/v1/me/games/"
gamesPage = s.get(endpoint, headers = headers).json()
games = gamesPage['results']
nextPage = gamesPage['next']

The following cell fetches information for the remaining games. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [ ]:
attempts = 0
max_attempts = 10
while nextPage != None: # to avoid timeout, will fix later
    response = s.get(nextPage,  headers = headers)
    # If not rate limited, continue
    if response.status_code != 429:
        attempts = 0
        print(nextPage)
        gamesPage = response.json()
        games.extend(gamesPage['results'])
        nextPage = gamesPage['next']
    # If rate limited, wait and then start again
    else:
        print("Rate limit reached. Backing off before retrying query. Please wait...")
        attempts += 1
        if attempts >= max_attempts:
            print("Too many attempts.")
            break
        else:
            time.sleep((2 ** attempts) + random.random())

In [ ]:
## Writing list of games to .json file
gamesJson = json.dumps(games, indent = 2)
f = open('raw_stats.json', 'w', encoding="utf-8")
f.write(gamesJson)
f.close()